(1) 학습/테스트 데이터 셋 분리하지 않고 예측

In [2]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

iris_data = load_iris()
dt_clf = DecisionTreeClassifier()

train_data = iris_data.data
train_label = iris_data.target

# 학습 수행
dt_clf.fit(train_data,train_label)

# 테스트
pred = dt_clf.predict(train_data)
print('예측 정확도:',accuracy_score(train_label,pred))

예측 정확도: 1.0


(2) 학습/테스트 데이터 셋 분리하고 예측

In [9]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

iris_data = load_iris()
dt_clf = DecisionTreeClassifier()

# 학습/테스트 분할(split)
X_train, X_test, y_train, y_test = train_test_split(iris_data.data,
                                                    iris_data.target,
                                                    test_size=0.2,
                                                    random_state=4)
print(y_train)

[1 0 2 0 1 2 2 1 1 0 2 0 1 0 2 0 0 1 1 2 0 1 2 2 1 1 0 1 2 1 0 1 0 1 2 1 2
 1 0 2 2 0 1 2 0 2 1 2 1 0 2 1 2 0 2 1 2 1 2 1 1 2 1 1 2 1 1 0 2 0 1 0 1 1
 1 1 0 2 2 1 1 1 0 0 2 2 0 0 0 2 0 0 2 2 1 0 0 0 2 1 0 0 2 1 2 0 0 2 1 1 1
 2 2 1 2 1 1 2 2 2]


In [10]:
# 학습 수행
dt_clf.fit(X_train,y_train)

# 테스트
pred = dt_clf.predict(X_test)
print('예측 정확도:',accuracy_score(y_test,pred))

예측 정확도: 0.9666666666666667


넘파이 ndarray뿐만 아니라 판다스 DataFrame/Series도 train_test_split으로 분할 가능

In [11]:
import pandas as pd

iris_df = pd.DataFrame(iris_data.data,columns=iris_data.feature_names)
iris_df['target'] = iris_data.target
iris_df.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0


In [24]:
# 피처 데이터프레임 반환(마지막 열 전까지, 마지막 열 제외)
feature_df = iris_df.iloc[:,:-1]

# 타깃 데이터프레임 반환
target_df = iris_df.iloc[:,-1:]

# 학습/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(feature_df,
                                                    target_df,
                                                    test_size=0.3,
                                                    random_state=4)

In [25]:
dt_clf=DecisionTreeClassifier()
dt_clf.fit(X_train,y_train)

pred = dt_clf.predict(X_test)
print('예측 정확도:',accuracy_score(y_test,pred))

예측 정확도: 0.9777777777777777


---

## 교차검증

- ### kFold 예제

In [32]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

iris = load_iris()
features = iris.data
label = iris.target

feature.shape

(150, 4)

In [33]:
dt_clf = DecisionTreeClassifier(random_state=156)

# 5개의 폴드 세트로 분리하는 KFold 객체 생성
kfold = KFold(n_splits=5)

# 폴드 세트별 정확도를 담을 리스트 객체 생성
cv_accuracy = []

In [34]:
kfold.split(features)

<generator object _BaseKFold.split at 0x00000248A75D46C8>

In [35]:
for train_index, test_index in kfold.split(features):
    print(train_index,test_index)

[ 30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47
  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 1

In [37]:
import numpy as np
for train_index, test_index in kfold.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    dt_clf.fit(X_train,y_train)
    pred = dt_clf.predict(X_test)
    
    acc = np.round(accuracy_score(y_test,pred),3) 
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    
    print('정확도 : %f, 학습데이터크기 : %d, 검증데이터크기 : %d'
          %(acc,train_size,test_size))
    cv_accuracy.append(acc)

# 교차검증 평가(평균)
print('평균 검증 정확도 : ',np.mean(cv_accuracy))

정확도 : 1.000000, 학습데이터크기 : 120, 검증데이터크기 : 30
정확도 : 0.967000, 학습데이터크기 : 120, 검증데이터크기 : 30
정확도 : 0.867000, 학습데이터크기 : 120, 검증데이터크기 : 30
정확도 : 0.933000, 학습데이터크기 : 120, 검증데이터크기 : 30
정확도 : 0.733000, 학습데이터크기 : 120, 검증데이터크기 : 30
평균 검증 정확도 :  0.9


- ### Stratifed K 폴드 교차 검증
    - 불균형한 분포도를 가진 레이블(결정 클래스) 데이터 집합을 위한 K폴드 방식

### <b>불균형한 데이터 문제</b>
- 관심 대상 데이터가 상대적으로 매우 적은 비율로 나타나는 데이터 문제
- 분류 문제인 경우 : 클래스들이 균일하게 분포하지 않은 문제를 의미
    - 회귀 문제인 경우 : 극단값이 포함되어 있는 "치우친"데이터 사례
    
#### 우회/극복하는 방법
- 데이터 추가 확보
- Re- Sampling
    - Under-sampling(과소표집)
        - 다른 클래스에 비하여 상대적으로 많이 나타나는 클래스의 개수를 줄임
        - 균형은 유지할 수 있으나 유용한 정보에 대한 손실이 있을 수 있음
    - Over-sampling(과대표집)
        - 상대적으로 적게 나타나는 클래스의 데이터를 복제하여 데이터의 개수를 늘림
        - 정보 손실은 없이 학습 성능은 높아지는 반면, 과적합의 위험이 있음
        - 이를 회피하기 위해서 SMOTE와 같이 임의의 값을 생성하여 추가하는 방법 사용

In [38]:
import pandas as pd

iris_df = pd.DataFrame(iris_data.data,columns=iris_data.feature_names)
iris_df['label'] = iris_data.target
iris_df.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0


In [41]:
# 3개 폴드를 구성
kfold = KFold(n_splits=3)
n = 0
for train_index, test_index in kfold.split(iris_df):
    n += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test =  iris_df['label'].iloc[test_index]
    print("[교차검증 : %d]"%n)
    print("  학습용 : \n",label_train.value_counts())
    print("  검증용 : \n",label_test.value_counts())
#     X_train, X_test = features[train_index], features[test_index]
#     y_train, y_test = label[train_index], label[test_index]

#     dt_clf.fit(X_train,y_train)
#     pred = dt_clf.predict(X_test)

[교차검증 : 1]
  학습용 : 
 1    50
2    50
Name: label, dtype: int64
  검증용 : 
 0    50
Name: label, dtype: int64
[교차검증 : 2]
  학습용 : 
 0    50
2    50
Name: label, dtype: int64
  검증용 : 
 1    50
Name: label, dtype: int64
[교차검증 : 3]
  학습용 : 
 0    50
1    50
Name: label, dtype: int64
  검증용 : 
 2    50
Name: label, dtype: int64


In [47]:
dt_clf = DecisionTreeClassifier(random_state=156)

# 5개의 폴드 세트로 분리하는 KFold 객체 생성
kfold = KFold(n_splits=3)

# 폴드 세트별 정확도를 담을 리스트 객체 생성
cv_accuracy = []

n = 0

for train_index, test_index in kfold.split(iris_df):
    X_train, X_test = feature[train_index], feature[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    dt_clf.fit(X_train,y_train)
    pred = dt_clf.predict(X_test)
    n+=1
    
    acc = np.round(accuracy_score(y_test,pred),3) 
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    
    print('%d \n정확도 : %f, 학습데이터크기 : %d, 검증데이터크기 : %d'
          %(n,acc,train_size,test_size))
    cv_accuracy.append(acc)

# 교차검증 평가(평균)
print('평균 검증 정확도 : ',np.mean(cv_accuracy))

1 
정확도 : 0.000000, 학습데이터크기 : 100, 검증데이터크기 : 50
2 
정확도 : 0.000000, 학습데이터크기 : 100, 검증데이터크기 : 50
3 
정확도 : 0.000000, 학습데이터크기 : 100, 검증데이터크기 : 50
평균 검증 정확도 :  0.0


In [49]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)
n=0

# 주어져있는 데이터의 분포를 고려
for train_index, test_index in skf.split(iris_df,iris_df['label']):
    n += 1
    
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print("[교차검증 %d]"% n)
    print("학습용 레이블 분포 : \n", label_train.value_counts())
    print("검증용 레이블 분포 : \n", label_test.value_counts())


[교차검증 1]
학습용 레이블 분포 : 
 2    34
0    33
1    33
Name: label, dtype: int64
검증용 레이블 분포 : 
 0    17
1    17
2    16
Name: label, dtype: int64
[교차검증 2]
학습용 레이블 분포 : 
 1    34
0    33
2    33
Name: label, dtype: int64
검증용 레이블 분포 : 
 0    17
2    17
1    16
Name: label, dtype: int64
[교차검증 3]
학습용 레이블 분포 : 
 0    34
1    33
2    33
Name: label, dtype: int64
검증용 레이블 분포 : 
 1    17
2    17
0    16
Name: label, dtype: int64


In [52]:
# StratifiedKFold를 이용해 붗꽃 데이터 교차 검증

dt_clf = DecisionTreeClassifier(random_state=156)

# 3개의 폴드 세트로 분리하는 StratifiedKFold 객체 생성
skfold = StratifiedKFold(n_splits=3)

# 폴드 세트별 정확도를 담을 리스트 객체 생성
cv_accuracy = []

n = 0

for train_index, test_index in skfold.split(features,label):
    X_train, X_test = feature[train_index], feature[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    # 학습 및 예측
    dt_clf.fit(X_train,y_train)
    pred = dt_clf.predict(X_test)
    
    # 반복 시 마다 정확도 측정
    n+=1
    
    acc = np.round(accuracy_score(y_test,pred),3) 
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('%d \n정확도 : %f, 학습데이터크기 : %d, 검증데이터크기 : %d'
          %(n,acc,train_size,test_size))
    cv_accuracy.append(acc)

# 교차검증 평가(평균)
print('## 평균 검증 정확도 : ',np.mean(cv_accuracy))

1 
정확도 : 0.980000, 학습데이터크기 : 100, 검증데이터크기 : 50
2 
정확도 : 0.940000, 학습데이터크기 : 100, 검증데이터크기 : 50
3 
정확도 : 0.980000, 학습데이터크기 : 100, 검증데이터크기 : 50
## 평균 검증 정확도 :  0.9666666666666667


### Stratified K 폴드의 경우
- 회귀에서는 지원되지 않음
    - 이유 : 회귀의 결정값은 이산값 형태의 레이블이 아니라 연속된 숫자값이기 때문에
    - 결정값별로 분포를 정하는 의미가 없기 때문
    
### 교차검증을 보다 간편하게
- <b>cross_val_score() 함수</b>  
estimator : Classifier 또는 Regressor  
X : feature dataset  
y : label dataset  
scoring : 예측 성능 평가 지표 측정값을 담은 배열  
cv : 교차검증 폴드 수   

### 붓꽃 자료를 3개 폴드로 분할하여 학습 및 검증

In [54]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.datasets import load_iris
import numpy as np

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)
features = iris.data # 
label = iris.target

scores = cross_val_score(dt_clf, features, label, scoring='accuracy',cv=3)
print('교차 검증별 정확도 : ',scores)
print('평균 검증 정확도',np.round(np.mean(scores),4))


교차 검증별 정확도 :  [0.98 0.94 0.98]
평균 검증 정확도 0.9667


---

## 교차 검증과 최적의 하이퍼파라미터 튜닝을 한번에  

### 사이킷런의 GridSearchCV클래스
- estimator : classfier, regressor, pipeline (알고리즘)
- param_grid : key + list 값을 가지는 딕셔너리
- scoring : 예측 성능을 측정할 평가 방법
- cv : 교차 검증을 위해 분할되는 학습/테스트 세트의 개수
- refit : 최적의 하이퍼 파라미터를 찾은 뒤 입력된 esimator객체를 해당 하이퍼 파라미터로 재학습 여부 (defalte : True)


In [63]:
# GridSearchCV를 이용해 결정 트리 알고리즘의 여러가지
#최적화 파라미터를 순차적으로 적용해서 붓꽃 데이터 예측분석

from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn. metrics import accuracy_score

iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data,
                                                   iris.target,
                                                   test_size=0.2,
                                                   random_state=121)
# 알고리즘 생성
dt_clf = DecisionTreeClassifier()

# 과대 적합을 방지하기 위한 방법 
# 하이퍼파라미터는 딕셔너리 형식으로 지정
# key : 결정트리의 하이퍼파라미터
# value : 하이퍼파라미터의 값
parameters = {'max_depth':[1,2,3],'min_samples_split':[2,3]}

In [65]:
# 
grid_tree = GridSearchCV(dt_clf, param_grid=parameters, cv=3, refit=True,
            return_train_score=True)
grid_tree.fit(X_train, y_train)

scores_df = pd.DataFrame(grid_tree.cv_results_)

In [66]:
scores_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,0.001658,0.000467,0.001664,0.000941,1,2,"{'max_depth': 1, 'min_samples_split': 2}",0.700,0.7,0.70,0.700000,1.110223e-16,5,0.7000,0.7000,0.7000,0.700000,1.110223e-16
1,0.001671,0.000937,0.001326,0.001249,1,3,"{'max_depth': 1, 'min_samples_split': 3}",0.700,0.7,0.70,0.700000,1.110223e-16,5,0.7000,0.7000,0.7000,0.700000,1.110223e-16
2,0.001000,0.000005,0.001014,0.000829,2,2,"{'max_depth': 2, 'min_samples_split': 2}",0.925,1.0,0.95,0.958333,3.118048e-02,3,0.9750,0.9375,0.9625,0.958333,1.559024e-02
3,0.000996,0.000003,0.000666,0.000471,2,3,"{'max_depth': 2, 'min_samples_split': 3}",0.925,1.0,0.95,0.958333,3.118048e-02,3,0.9750,0.9375,0.9625,0.958333,1.559024e-02
4,0.001328,0.000472,0.000676,0.000478,3,2,"{'max_depth': 3, 'min_samples_split': 2}",0.975,1.0,0.95,0.975000,2.041241e-02,1,0.9875,0.9625,0.9875,0.979167,1.178511e-02
5,0.003330,0.001701,0.000669,0.000473,3,3,"{'max_depth': 3, 'min_samples_split': 3}",0.975,1.0,0.95,0.975000,2.041241e-02,1,0.9875,0.9625,0.9875,0.979167,1.178511e-02


In [68]:
scores_df[['params','mean_test_score','rank_test_score']]

,params,mean_test_score,rank_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1


In [70]:
# 최고 성능을 가지는 파라미터 조합 및 예측 성능 1위 값 출력
print('최적 파라미타 : ',grid_tree.best_params_)
print('최고 정확도 : ',grid_tree.best_score_)

최적 파라미타 :  {'max_depth': 3, 'min_samples_split': 2}
최고 정확도 :  0.975


In [71]:
# GridSearchCV가 최적 성능을 나타내는 하이퍼 파라미터로 Estimator를 학습하고
# best_estimator_ 로 저장
best_dt = grid_tree.best_estimator_

# best_estimator_는 이미 최적 학습이 됐으므로 별도 학습 필요없이 바로 예측 가능
pred = best_dt.predict(X_test)
accuracy_score(y_test,pred)

0.9666666666666667

1. 학습데이터와 테스트데이터로 나눈다. -> train_test_split
2. 반복해서 성능을 테스트하는 법 -> 교차검증
3. 교차검증을 편하기 하기 -> cross_val_score()
4. 최적 하이퍼 파라미터를 튜닝하여 교차검증까지 해주는 함수 -> GridSearchCV